In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

In [2]:
# Load NSL-KDD Dataset
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", 
    "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", 
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", 
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", 
    "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", 
    "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", 
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", 
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", 
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label", "difficulty_level"
]

In [3]:
file_path = 'nsl-kdd/KDDTrain+.txt' 
df = pd.read_csv(file_path, header=None, names=column_names)

In [4]:
# One-Hot Encoding for Categorical Columns
categorical_columns = ['protocol_type', 'service', 'flag']
df = pd.get_dummies(df, columns=categorical_columns)

In [5]:
# Save expected columns for feature alignment
train_columns = df.drop(['label', 'difficulty_level'], axis=1).columns.tolist()
joblib.dump(train_columns, 'train_columns.pkl')

['train_columns.pkl']

In [6]:
# Encode Labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [7]:

# Save all possible classes
all_classes = np.unique(df['label'])

In [8]:
# Prepare Features and Labels
X = df.drop(['label', 'difficulty_level'], axis=1)
y = df['label']

In [9]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
# Remove extremely low-sample classes before resampling
min_samples_threshold = 5  # Adjust as needed
filtered_classes = y_train.value_counts()[y_train.value_counts() >= min_samples_threshold].index
X_train = X_train[y_train.isin(filtered_classes)]
y_train = y_train[y_train.isin(filtered_classes)]

In [11]:
# Apply SMOTE to Balance Classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [12]:

# Ensure `y_train_balanced` contains all possible classes
missing_classes = np.setdiff1d(all_classes, np.unique(y_train_balanced))
if len(missing_classes) > 0:
    for cls in missing_classes:
        X_train_balanced = np.vstack([X_train_balanced, np.zeros((1, X_train_balanced.shape[1]))])
        y_train_balanced = np.append(y_train_balanced, cls)

In [13]:

# Standardization
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(pd.DataFrame(X_train_balanced, columns=train_columns))
X_test = scaler.transform(pd.DataFrame(X_test, columns=train_columns))
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [14]:
# Train XGBoost Model
clf = XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_balanced, y_train_balanced)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
# Evaluate Model
y_pred = clf.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  191     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [    0     5     0     0     0     0     0     0     0     0     0     1
      0     0     0     0     0     0     0     0     0]
 [    0     1     0     0     0     0     0     0     0     0     0     1
      0     0     0     0     0     0     0     0     0]
 [    0     0     0    11     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     2     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0   713     0     0     0     0     7     0
      0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     4     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     1     0     0    

C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [16]:
# Save Model
joblib.dump(clf, 'ids_model.pkl')
print("Model saved successfully!")

Model saved successfully!
